
# Range Box Detection + MetaTrader 5 Login (Python Notebook)

This notebook helps you:
- **Log in to MetaTrader 5** via the official Python package
- **Download OHLC data** for a symbol/timeframe
- **Detect ranging sections** where *high–low* stays within a threshold over a rolling lookback (like your TradingView script)
- **(Optional)** Plot the ranges as shaded boxes

> **Note:** To connect to MT5, you must have the **MetaTrader 5 terminal installed** on your machine and the **`MetaTrader5`** Python package available. If the terminal isn't auto-detected, you'll need to pass its full path to `mt5_login()`.


In [1]:
from idlelib.iomenu import errors

from mt5.pip_value import pip_value_usd

# If needed, install packages (uncomment these lines on your machine)
# !pip install MetaTrader5 pandas matplotlib

instrument_map = {
    "XAUUSD":    ("XAUUSD+", 1.5),
    "BTCUSD":    ("BTCUSD", 50),
    "ETHUSD":    ("ETHUSD", 5),
    "NAS100":    ("NAS100", 5),
    "GBPUSD":    ("GBPUSD+", 0.00030),
    "EURUSD":    ("EURUSD+", 0.00030),
    "USDJPY":    ("USDJPY+", 1)
}

from mt5.mt5_login import mt5_login
import pandas as pd

try:
    import MetaTrader5 as mt5
    HAS_MT5 = True
except Exception as e:
    HAS_MT5 = False
    print("MetaTrader5 package is not available in this environment. "
          "Install it with `pip install MetaTrader5` on your machine to use MT5 features.")




success = mt5_login(
    login=1,
    password='1',
    server='VantageFX-Demo',
    path=r'C:\Program Files\MetaTrader 5\terminal64.exe',
    portable=False)
if not success:
    raise SystemExit("Could not connect to MT5. Check credentials/path and try again.")


Terminal connected: MetaTrader 5 | build: 5200
Account: 10981514 | Server: VantageFX-Demo | Balance: 1172348.83


In [ ]:
from twilio.send_notification import send_sms

## MetaTrader 5 Utilities

## Fetch market data and detect ranges **COMMENTED OUT**

In [ ]:
# from mt5.ranges import merge_consecutive_ranges, find_body_ranges
# from mt5.fetch_rates import mt5_fetch_rates
#
#
# def fetch_market_data(instrument: str, timeframe: int, bars: int) -> pd.DataFrame:
#
#     # --- 2) Fetch data (choose timeframe) ---
#     tframe = mt5.TIMEFRAME_M5  # or mt5.TIMEFRAME_M1
#     lookback = 4  # if your data is M5; use 50 if your data is M1
#     # instrument = 'BTCUSD'  # 'GBPUSD+', 'USDJPY+', 'EURUSD+', 'XAUUSD+', 'NAS100', 'BTCUSD'
#     df = mt5_fetch_rates(instrument, timeframe,bars)  # 5520 bars ~ 46 days for M5, ~ 230 days for M1
#     # --- 3) Choose lookback based on timeframe (example: 5m -> 10 bars; 1m -> 50 bars) ---
#     # Set range_size based on instrument
#     if instrument.startswith('BTCUSD'):
#         range_size = 50
#     elif instrument.startswith('NAS100'):
#         range_size = 5
#     elif instrument.startswith('XAUUSD'):
#         range_size = 1.5
#     elif instrument.startswith('GBPUSD'):
#         range_size = 0.00030
#     elif instrument.startswith('USDJPY'):
#         range_size = 0.003
#     else:
#         range_size = 1  # default for other instruments
#     # --- 4) Detect ranges ---
#     ranges = find_body_ranges(df, lookback=lookback, range_size=range_size)
#     if ranges:
#         ranges_df = pd.DataFrame(ranges)
#         merged_df = merge_consecutive_ranges(ranges_df)
#         print(f"Found {len(merged_df)} ranges for {instrument}")
#         # print(ranges_df[["start_idx", "end_idx", "start_ts", "end_ts", "top", "bottom", "mid", "range_value",
#         #                  "duration_bars"]].to_string())
#         return ranges_df
#     # --- 5) Plot (optional) ---
#     # plot_ranges(df, ranges, title=f"Detected Ranges (lookback={lookback}, size=${range_size})")
#
#
#


## Fetch market data

In [ ]:
from mt5.fetch_rates import mt5_fetch_rates


# Fetch data for a specific instrument and timeframe
instrument = 'XAUUSD+'
timeframe = mt5.TIMEFRAME_M5  # or mt5.TIMEFRAME_M1
bars = 5520  # ~46 days for M5, ~230 days for M1
rates_data = mt5_fetch_rates(instrument, timeframe, 5520)
if rates_data is not None:
    print("data length " + str(len(rates_data)))
    # print(pd.DataFrame(data)[["open", "high", "low", "close","rsi"]].to_string())
    # print(data.head)
    # print(f"Found {len(merged_df)} ranges for {instrument}")
    print(rates_data.head(3200).to_string())
    print(rates_data.tail().to_string())

else:
    print("No data fetched for the specified range.")

## Calculate Body Ranges and merged Ranges

In [ ]:
from mt5.ranges import find_body_ranges
from mt5.merge_ranges import merge_ranges
import pandas as pd


body_range  = find_body_ranges(rates_data, 4,1.5)
print(body_range.head().to_string())
if body_range is not None and not body_range.empty:
    body_range['date'] = pd.to_datetime(body_range['time'], format='%d-%m-%Y %H:%M', errors='coerce').dt.date
    merged_ranges = merge_ranges(body_range)
    start_date = body_range['date'].min()
    end_date = body_range['date'].max()
    print("Start date:", start_date)
    print("End date:", end_date)
    print(f"range count before merge: {len(body_range)}")
    print(f"range count after merge: {len(merged_ranges)}")
else:
    print("No body ranges found for the specified data.")


In [ ]:
# body_range =body_range.iloc[::-1]  # Reverse the order of the DataFrame
body_range

In [ ]:
from mt5.fetch_rates import mt5_fetch_rates_range

data_in_range = mt5_fetch_rates_range("BTCUSD", mt5.TIMEFRAME_M5, "1/8/2025", "15/8/2025")

In [ ]:
body_range  = find_body_ranges(data_in_range, 4,50)
body_range['date'] = pd.to_datetime(body_range['time'], format='%d-%m-%Y %H:%M').dt.date
merged_ranges = merge_ranges(body_range)
start_date = body_range['date'].min()
end_date = body_range['date'].max()
print("Start date:", start_date)
print("End date:", end_date)
print(f"range count before merge: {len(body_range)}")
print(f"range count after merge: {len(merged_ranges)}")

In [ ]:
from mt5.lot_calculator import calc_lot_size
!pip install fastapi uvicorn nest_asyncio
from fastapi import FastAPI, HTTPException
from mt5.order import OrderResponseModel, serialize_mt5_result, retcode_name, OrderRequest, map_fill, place_order, \
    map_time
import nest_asyncio
nest_asyncio.apply()
import json

from fastapi.middleware.cors import CORSMiddleware
from mt5.ranges import find_body_ranges
from mt5.merge_ranges import merge_ranges
from fastapi import FastAPI, Query
from mt5.fetch_rates import mt5_fetch_rates
from urllib.parse import unquote

import uvicorn

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/fetch_symbol_map")
def fetch_symbol_map():
    """
    Fetch the instrument map with symbols and their corresponding range sizes.
    """
    return {
        "data": instrument_map
    }

@app.get("/calculate_lots")
def calculate_lots(
        symbol: str = Query(...),
        risk_amount: int = Query(...),
        sl_pips: int = Query(...),
):
    try:
        symbol = instrument_map[symbol.upper().strip()][0]
    except Exception as e:
        print(f"Error fetching symbol: {e}")
    lots = calc_lot_size(symbol,risk_amount, sl_pips)
    print(f"Lot sizes: {lots}")

    return {
        "symbol": symbol,
        "lots": lots,
        "risk_amount": risk_amount,
        "sl_pips": sl_pips
    }

@app.get("/fetch_tick_data")
def fetch_tick_data(
    symbol: str = Query(...)
):
    try:
        symbol = instrument_map[symbol.strip()][0]
    except Exception as e:
        print(f"Error fetching symbol: {e}")

    print(f"Fetching tick data for symbol: {symbol}")
    tick_info = mt5.symbol_info(symbol)
    # print(json.dumps(mt5.symbol_info(symbol)._asdict(), indent=2))
    pip_value = pip_value_usd(tick_info)
    print(f"PIP value for {symbol}: {pip_value}")
    if tick_info is not None:
        return {
            "data": {
                "symbol": symbol,
                "bid": tick_info.bid,
                "bidHigh": tick_info.bidhigh,
                "bidLow": tick_info.bidlow,
                "ask": tick_info.ask,
                "askHigh": tick_info.askhigh,
                "askLow": tick_info.asklow,
                "digits": tick_info.digits,
                "spread": tick_info.spread,
                "point": tick_info.point,
                "pipValue":pip_value,
                "tradeTickSize": tick_info.trade_tick_size,
                "tradeTickValueProfit": tick_info.trade_tick_value_profit,
                "tradeTickValue": tick_info.trade_tick_value,
                "volumeStep": tick_info.volume_step,
                "volumeMin": tick_info.volume_min,
                "volumeMax": tick_info.volume_max
            }
        }
    else:
        raise HTTPException(status_code=404, detail=f"Tick data for {symbol} not found.")


@app.get("/fetch_data")
def fetch_all_data(
    tf: int = Query(5),
    candles: int = Query(5520)
):
    all_results = []
    for inst, (mapped_inst, range_size) in instrument_map.items():
        print(f"instrument is {mapped_inst}")
        rates_data = mt5_fetch_rates(mapped_inst, tf, candles)
        if rates_data is not None:
            body_range = find_body_ranges(rates_data, 4, range_size)
            merged_ranges = merge_ranges(body_range)
            merged_ranges["instrument"] = inst
            all_results.append(merged_ranges)
    if all_results:
        result_df = pd.concat(all_results, ignore_index=True)
        return {
            "data_length": len(result_df),
            "data": result_df.to_dict(orient="records")
        }
    else:
        return {"error": "No data fetched for any instrument."}

@app.get("/fetch_data")
def fetch_data(
    instrument: str = Query(...),
    timeframe: int = Query(5),
    bars: int = Query(5520)
):
    symbol = unquote(instrument)
    print(f"Fetching data for {symbol}")
    range_size = 1.5  # Default range size for XAUUSD
    if instrument.startswith('BTCUSD'):
        range_size = 50
    if instrument.startswith('ETHUSD'):
        range_size = 50
    elif instrument.startswith('NAS100'):
        range_size = 5
    elif instrument.startswith('XAUUSD'):
        instrument = "XAUUSD+"
        range_size = 1.5
    elif instrument.startswith('GBPUSD'):
        instrument = "GBPUSD+"
        range_size = 0.00030
    elif instrument == "EURUSD":
        instrument = "EURUSD+"
        range_size = 0.00030
    elif instrument.startswith('USDJPY'):
        instrument = "USDJPY+"
    else:
        raise ValueError("Unsupported instrument. Please use one of the supported instruments.")

    print(f"instrument is {instrument}")
    rates_data = mt5_fetch_rates(instrument, timeframe, bars)
    body_range  = find_body_ranges(rates_data, 4,range_size)
    merged_ranges = merge_ranges(body_range)
    if rates_data is not None:
        return {
            "data_length": len(merged_ranges),
            "data": merged_ranges.to_dict(orient="records")
        }
    else:
        return {"error": "No data fetched for the specified range."}

@app.post("/orders", response_model=OrderResponseModel)
def create_order(req: OrderRequest):
    type_filling = map_fill(req.type_filling)
    type_time = map_time(req.type_time)

    result = place_order(
        symbol=req.symbol,
        side=req.side,
        volume=req.volume,
        kind=req.kind,
        price=req.price,
        stoplimit=req.stoplimit,
        deviation=req.deviation,
        sl_points=req.sl_points,
        tp_points=req.tp_points,
        sl_price=req.sl_price,
        tp_price=req.tp_price,
        magic=req.magic,
        comment=req.comment,
        do_order_check=req.do_order_check,
        type_filling=type_filling,
        type_time=type_time,
        expiration=req.expiration,
    )

    if not result.ok:
        payload = OrderResponseModel(
            ok=result.ok,
            retcode=result.retcode,
            retcode_meaning=retcode_name(result.retcode),
            comment=result.comment,
            result=serialize_mt5_result(result.result),
        )
        # Convert to dict for JSON serialization
        raise HTTPException(status_code=400, detail=payload.dict())

    payload = OrderResponseModel(
        ok=result.ok,
        retcode=result.retcode,
        retcode_meaning=retcode_name(result.retcode),
        comment=result.comment,
        result=serialize_mt5_result(result.result),
    )
    # For MT5 rejections (positive retcodes but not DONE), still return 200 with details
    return payload

uvicorn.run(app, host="0.0.0.0", port=8000)

instrument is XAUUSD+
Fetching rates for symbol with updated function: XAUUSD+
instrument is BTCUSD
Fetching rates for symbol with updated function: BTCUSD
instrument is ETHUSD
Fetching rates for symbol with updated function: ETHUSD
instrument is NAS100
Fetching rates for symbol with updated function: NAS100
instrument is GBPUSD+
Fetching rates for symbol with updated function: GBPUSD+
instrument is EURUSD+
Fetching rates for symbol with updated function: EURUSD+
instrument is USDJPY+
Fetching rates for symbol with updated function: USDJPY+
INFO:     10.211.55.3:53911 - "GET /fetch_data?instrument=XAUUSD+ HTTP/1.1" 200 OK
